# Parsing top-down direzionale (caso generale)

In [17]:
from liblet import Production, Grammar, Derivation, ProductionGraph, Queue, Stack, dod2table
from ipywidgets import interactive, IntSlider

## Imitando le derivazioni leftmost

In [2]:
# fig. 6.1, pag. 165

G = Grammar.from_string("""
S -> a B | b A
A -> a | a S | b A A
B -> b | b S | a B B
""")
G

Grammar(N={A, B, S}, T={a, b}, P=(S -> a B, S -> b A, A -> a, A -> a S, A -> b A A, B -> b, B -> b S, B -> a B B), S=S)

In [3]:
def derivation_up_to(n):
    leftmost_prods = [0, 7, 5, 5]
    d = Derivation(G)
    for prod in leftmost_prods[:n]: d = d.leftmost(prod)
    display(ProductionGraph(d))

# quel che segue è un truccaccio di visualizzazione, non badateci    
    
ui = interactive(derivation_up_to, n = IntSlider(min = 0, max = 4, value = 0))
ui.children[-1].layout.height = '300px'
ui

interactive(children=(IntSlider(value=0, description='n', max=4), Output(layout=Layout(height='300px'))), _dom…

In [4]:
# non presente nel libro

G = Grammar.from_string("""
S -> a B C
B -> a B | b
C -> a
""")

def derivation_up_to(n):
    leftmost_prods = [0, 1, 2, 3]
    d = Derivation(G)
    for prod in leftmost_prods[:n]: d = d.leftmost(prod)
    display(ProductionGraph(d))

# quel che segue è un truccaccio di visualizzazione, non badateci    
    
ui = interactive(derivation_up_to, n = IntSlider(min = 0, max = 4, value = 0))
ui.children[-1].layout.height = '300px'
ui

interactive(children=(IntSlider(value=0, description='n', max=4), Output(layout=Layout(height='300px'))), _dom…

## Aggiornare la pila di predizione e la derivazione

In [5]:
def update_stack(stack, P):
    copy = stack.copy()
    for X in reversed(P.rhs): copy.push(X)
    return copy

def update_deriv(deriv, P):
    return deriv.leftmost(deriv.G.P.index(P))

In [10]:
# mostra le descrizoni istantanee nella coda/pila

def show_instdescrs(store):
    for deriv, stack, rest in store:
        print([p for p, q in deriv.steps()], ''.join(reversed(list(stack))), rest)
    print('-' * 60)

## Calcolare le prossime descrizioni istantanee

In [12]:
def next_instdescrs(curr):
    deriv, stack, rest = curr
    top = stack.pop()
    if top == rest[0] == '#': return [(deriv, None, None)]
    if top in G.T:
        # MATCH
        if top == rest[0]: 
            return [
                (deriv, stack.copy(), rest[1:])
            ]
    else:
        # PREDICT
        return [
            (update_deriv(deriv, P), update_stack(stack, P), rest)
            for P in filter(Production.such_that(lhs = top), G.P)
        ]
    return []

## Nondeterminismo

In [24]:
# fig. 6.6, pag. 171

G = Grammar.from_string("""
S -> A B | D C 
A -> a | a A
B -> b c | b B c 
D -> a b | a D b 
C -> c | c C
""")
G

word = 'aabc'

### Breadth first

In [25]:
def breadth_first(G, word, show = True, max_steps = -1):
    q = Queue()
    q.enqueue((Derivation(G), Stack(['#', G.S]), word + '#'))
    derivations = []
    steps = 0
    while q:
        if steps > max_steps > -1: break
        steps += 1
        if show: show_instdescrs(q)
        curr = q.dequeue()
        for nxt in next_instdescrs(curr): 
            deriv, stack, rest = nxt
            if rest is None:
                derivations.append(deriv)
            else:
                q.enqueue(nxt)
    return derivations

breadth_first(G, word, True)

[] S# aabc#
------------------------------------------------------------
[0] AB# aabc#
[1] DC# aabc#
------------------------------------------------------------
[1] DC# aabc#
[0, 2] aB# aabc#
[0, 3] aAB# aabc#
------------------------------------------------------------
[0, 2] aB# aabc#
[0, 3] aAB# aabc#
[1, 6] abC# aabc#
[1, 7] aDbC# aabc#
------------------------------------------------------------
[0, 3] aAB# aabc#
[1, 6] abC# aabc#
[1, 7] aDbC# aabc#
[0, 2] B# abc#
------------------------------------------------------------
[1, 6] abC# aabc#
[1, 7] aDbC# aabc#
[0, 2] B# abc#
[0, 3] AB# abc#
------------------------------------------------------------
[1, 7] aDbC# aabc#
[0, 2] B# abc#
[0, 3] AB# abc#
[1, 6] bC# abc#
------------------------------------------------------------
[0, 2] B# abc#
[0, 3] AB# abc#
[1, 6] bC# abc#
[1, 7] DbC# abc#
------------------------------------------------------------
[0, 3] AB# abc#
[1, 6] bC# abc#
[1, 7] DbC# abc#
[0, 2, 4] bc# abc#
[0, 2, 5] bBc# 

[S -> A B -> a A B -> a a B -> a a b c]

### Depth first

In [27]:
def depth_first(G, word, show = True, max_steps = -1):
    s = Stack()
    s.push((Derivation(G), Stack(['#', G.S]), word + '#'))
    derivations = []
    steps = 0
    while s:
        if steps > max_steps > -1: break
        steps += 1
        if show: show_instdescrs(s)
        curr = s.pop()
        for nxt in next_instdescrs(curr): 
            deriv, stack, rest = nxt
            if rest is None:
                derivations.append(deriv)
            else:
                s.push(nxt)
    return derivations

depth_first(G, word, True)

[] S# aabc#
------------------------------------------------------------
[0] AB# aabc#
[1] DC# aabc#
------------------------------------------------------------
[0] AB# aabc#
[1, 6] abC# aabc#
[1, 7] aDbC# aabc#
------------------------------------------------------------
[0] AB# aabc#
[1, 6] abC# aabc#
[1, 7] DbC# abc#
------------------------------------------------------------
[0] AB# aabc#
[1, 6] abC# aabc#
[1, 7, 6] abbC# abc#
[1, 7, 7] aDbbC# abc#
------------------------------------------------------------
[0] AB# aabc#
[1, 6] abC# aabc#
[1, 7, 6] abbC# abc#
[1, 7, 7] DbbC# bc#
------------------------------------------------------------
[0] AB# aabc#
[1, 6] abC# aabc#
[1, 7, 6] abbC# abc#
[1, 7, 7, 6] abbbC# bc#
[1, 7, 7, 7] aDbbbC# bc#
------------------------------------------------------------
[0] AB# aabc#
[1, 6] abC# aabc#
[1, 7, 6] abbC# abc#
[1, 7, 7, 6] abbbC# bc#
------------------------------------------------------------
[0] AB# aabc#
[1, 6] abC# aabc#
[1, 7, 6] abb

[S -> A B -> a A B -> a a B -> a a b c]

## Ricorsione

In [35]:
# pag. 173

G = Grammar.from_string('S -> a | S b')
G

word = 'ab'

In [36]:
depth_first(G, word, True, 10)

[] S# ab#
------------------------------------------------------------
[0] a# ab#
[1] Sb# ab#
------------------------------------------------------------
[0] a# ab#
[1, 0] ab# ab#
[1, 1] Sbb# ab#
------------------------------------------------------------
[0] a# ab#
[1, 0] ab# ab#
[1, 1, 0] abb# ab#
[1, 1, 1] Sbbb# ab#
------------------------------------------------------------
[0] a# ab#
[1, 0] ab# ab#
[1, 1, 0] abb# ab#
[1, 1, 1, 0] abbb# ab#
[1, 1, 1, 1] Sbbbb# ab#
------------------------------------------------------------
[0] a# ab#
[1, 0] ab# ab#
[1, 1, 0] abb# ab#
[1, 1, 1, 0] abbb# ab#
[1, 1, 1, 1, 0] abbbb# ab#
[1, 1, 1, 1, 1] Sbbbbb# ab#
------------------------------------------------------------
[0] a# ab#
[1, 0] ab# ab#
[1, 1, 0] abb# ab#
[1, 1, 1, 0] abbb# ab#
[1, 1, 1, 1, 0] abbbb# ab#
[1, 1, 1, 1, 1, 0] abbbbb# ab#
[1, 1, 1, 1, 1, 1] Sbbbbbb# ab#
------------------------------------------------------------
[0] a# ab#
[1, 0] ab# ab#
[1, 1, 0] abb# ab#
[1, 1, 1, 0] ab

[]

### Un "truccaccio"

In [39]:
def depth_first_trick(G, word, show = True):
    s = Stack()
    s.push((Derivation(G), Stack(['#', G.S]), word + '#'))
    derivations = []
    while s:
        if show: show_instdescrs(s)
        curr = s.pop()
        for nxt in next_instdescrs(curr): 
            deriv, stack, rest = nxt
            if rest is None:
                derivations.append(deriv)
            else:
                if len(stack) > len(rest): continue
                s.push(nxt)
    return derivations

depth_first_trick(G, word, False)

[S -> S b -> a b]

# Homework

* verificare che `depth_first`, `breadth_first` e `depth_first_trick` "funzionino" anche in presenza di ε-regole,
  "aggiustando" le tre funzioni se necessario,
